In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import plotly.graph_objs as go
import plotly.offline as py
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, add_changepoints_to_plot
py.init_notebook_mode()

In [ ]:
df = pd.read_csv("/kaggle/input/corona0221202/time_series_19-covid-Confirmed.csv")
df.head()

In [ ]:
df_korea = df[df['Country/Region'] == 'South Korea']

In [ ]:
df_korea.head()

In [ ]:
df_korea = df_korea.T[4:]
df_korea = df_korea.reset_index().rename(columns={'index': 'date', 33:'confirmed'})

df_korea['date'] = pd.to_datetime(df_korea['date'])
df_korea.head()

In [ ]:
df_korea = df_korea.append([{'date': pd.to_datetime('2020-02-22'), 'confirmed': 433},
                           {'date':pd.to_datetime('2020-02-23'), 'confirmed':602},
                           {'date':pd.to_datetime('2020-02-24'),'confirmed':833},
                           {'date':pd.to_datetime('2020-02-25'),'confirmed':977},
                           {'date':pd.to_datetime('2020-02-26'),'confirmed':1146},
                           {'date':pd.to_datetime('2020-02-27'),'confirmed':1766},
                           {'date':pd.to_datetime('2020-02-28'),'confirmed':2337},
                            {'date':pd.to_datetime('2020-02-29'),'confirmed':3150},
                            {'date':pd.to_datetime('2020-03-01'),'confirmed':3736}
                           ],
                           ignore_index=True)
df_korea.tail()


In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
    x=df_korea.date,
    y=df_korea.confirmed,
    name ='corona in korea'))

In [ ]:
df_prophet = df_korea.rename(columns={'date':'ds','confirmed':'y'})
df_prophet.head()

In [ ]:
m = Prophet(
    changepoint_prior_scale=0.5,
    changepoint_range=0.98,
    yearly_seasonality=False,
    weekly_seasonality=False,
    daily_seasonality=True,
    seasonality_mode = 'additive')

m.fit(df_prophet)
future = m.make_future_dataframe(periods=7)
forecast = m.predict(future)
fig = plot_plotly(m, forecast)
py.iplot(fig)

In [ ]:
fig = m.plot(forecast)
a = add_changepoints_to_plot(fig.gca(), m, forecast)

In [ ]:
forecast[['ds','trend']].tail(20)